In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime
import jpholiday
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
# from optuna.integration import lightgbm as lgb
import lightgbm as lgb
from sklearn.metrics import  log_loss
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from datetime import date, timedelta
import sklearn.metrics
import optuna 
from dateutil.relativedelta import relativedelta
import pandas_profiling as pdp


%matplotlib inline

# 基準モデルの作成

In [2]:
FILE_PATH = '/Users/atsushisato/git_clone/ai_quest_season2/data/'

train_data = pd.read_csv(FILE_PATH + 'external/train.csv').rename(columns={'顧客ID':'uuid', 'クーポンID':'coupon_id', 'クーポン利用':'coupon_use_result'})
test_data = pd.read_csv(FILE_PATH + 'external/test.csv').rename(columns={'顧客ID':'uuid', 'クーポンID':'coupon_id'})
customer_df = pd.read_csv(FILE_PATH + 'interim/customer_df.csv')

# 特徴量作成

In [3]:
#特徴量作成関数（trainとtestで異なる処理が可能）
def create_feature_separate(dataframe):
    # issue 9/10 カテゴリ変数の数値変換 
    dataframe = category_to_int_conversion(dataframe)
    
    # issue 11 カテゴリ別商品単価
    dataframe['unit_price_category'] = dataframe.apply(lambda x: 
                                unit_price_category(x['purchase_product_num'], x['coupon_use'], x['purchase_price']),axis=1)
    # issue 16 総購入商品の平均
    dataframe['purchase_price_average'] = dataframe.apply(lambda x: 
                                purchase_price_average(x['all_purchase_price'], x['visits_frequency']),axis=1)   
    # issue 20 平均来店周期
    dataframe['visit_cycle'] = dataframe.apply(lambda x:
                                visit_cycle(x['visits_frequency'], x['pass_days']),axis=1)
    # issue 21 経過日数と来店頻度の比較
    dataframe['visit_cycle_flg'] = dataframe.apply(lambda x: 
                                visit_cycle_flg(x['visit_cycle'], x['pass_days']),axis=1) 
    # issue 22 総購入金額に占めるカテゴリ別の購入金額比率
    dataframe['ratio_category'] = dataframe.apply(lambda x:
                                ratio_category(x['all_purchase_price'], x['purchase_price']),axis=1)
    # issue 23 大人の数
    dataframe['adult_num'] = dataframe['fammily_num'] - dataframe['child_num']
    
    # issue 24 カテゴリ別平均購入金額
    dataframe['purchase_price_category_average'] = dataframe.apply(lambda x:
                                purchase_price_category_average(x['purchase_price'], x['purchase_product_num'],x['coupon_use']),axis=1)
    # issue 28 家族1人あたりの総購入金額
    dataframe['purchase_price_1person'] = dataframe.apply(lambda x:
                                purchase_price_1person(x['all_purchase_price'], x['fammily_num']),axis=1)
    # issue 29 1日あたりの購入金額
    dataframe['purchase_price_1day'] = dataframe.apply(lambda x :
                                purchase_price_1day(x['all_purchase_price'], x['pass_days']),axis=1)
    
    
    dataframe = dataframe.drop(['category_id', 'house_flg', 'marry_flg', 'age', 'visit_cycle_flg', 'category_id'],axis=1)
    return dataframe

In [4]:
#===================================================================================
#特徴量作成関数
# issue 9/10 カテゴリ変数の数値変換    
def category_to_int_conversion(dataframe):
    marry_dict = {'無回答':None, '独身':0, '既婚':1}
    dataframe['marry_flg'] = dataframe['marry'].apply(lambda x: marry_dict.get(x))
    
    dataframe['age'] = dataframe['age_range'].apply(lambda x: int(x[:2]))
    dataframe['category_id'] = dataframe['category_id'].apply(lambda x: int(x[1:]))
    
    dataframe = dataframe.drop(['age_range','marry','category'],axis=1)
    return dataframe

# issue 11 カテゴリ別商品単価
def unit_price_category(product_num, coupon_use, price):
    if coupon_use != 0:
        price = price + abs(coupon_use)
    
    if price != 0:
        return int(price / product_num)
    else:
        return 0
    
# issue 16 総購入商品の平均
def purchase_price_average(all_purchase_price, visits_frequency):
    if (all_purchase_price != 0) & (visits_frequency != 0):
        one_purchase_price =  int(all_purchase_price / visits_frequency)            
    else:
        one_purchase_price =  0
    return one_purchase_price

# issue 20 平均来店周期
def visit_cycle(visits_frequency, pass_days):
    all_days = 60
    if visits_frequency != 0:
        cycle = int((all_days - pass_days) / visits_frequency)
    else:
        cycle = 0
    return cycle

# issue 21 経過日数と来店頻度の比較
def visit_cycle_flg(visit_cycle, pass_days):
    if visit_cycle <= pass_days:
        return 1
    else:
        return 0
    
# issue 22 総購入金額に占めるカテゴリ別の購入金額比率
def ratio_category(all_purchase_price, purchase_price):
    if purchase_price != 0:
        ratio = purchase_price / all_purchase_price
    else:
        ratio = 0
    return ratio

# issue 24 カテゴリ別平均購入金額
def purchase_price_category_average(purchase_price, purchase_product_num, coupon_use):
    if purchase_product_num != 0:
        purchase_price = purchase_price + abs(coupon_use)
        average_purchase = int(purchase_price / purchase_product_num)
    else:
        average_purchase = 0
    return average_purchase
    
# issue 28 家族1人あたりの総購入金額
def purchase_price_1person(all_purchase_price, fammily_num):
    if all_purchase_price != 0:
        return int(all_purchase_price / fammily_num)
    else:
        return 0

# issue 29 1日あたりの購入金額
def purchase_price_1day(all_purchase_price, pass_days):
    if all_purchase_price != 0:
        all_days = 60
        use_days = all_days - pass_days + 1
        return int(all_purchase_price / use_days)
    else:
        return 0
    
# issue 19 欠損値削除（購入履歴はあるが来店履歴がないuuid）
def drop_missing_value(dataframe):
    drop_uuid = dataframe.query("(visits_frequency ==0 )& (all_purchase_price != 0)")['uuid'].unique()
    dataframe = dataframe.query("uuid not in @drop_uuid") 
    return dataframe

In [5]:
#train_dataとtest_dataの特徴量作成
train_data = pd.merge(train_data, customer_df, on=['uuid', 'coupon_id'],how='inner')
test_data = pd.merge(test_data, customer_df, on=['uuid', 'coupon_id'],how='inner')

#特徴量作成（train_dataとtest_dataで異なる処理が可能）
train_data = create_feature_separate(train_data)
test_data = create_feature_separate(test_data)
train_data = drop_missing_value(train_data)

In [6]:
# pdp.ProfileReport(train_data.query("coupon_use_result == 1"))

In [7]:
sorted(train_data.columns)

['adult_num',
 'all_purchase_price',
 'child_num',
 'coupon_id',
 'coupon_use',
 'coupon_use_result',
 'fammily_num',
 'income',
 'pass_days',
 'purchase_num',
 'purchase_price',
 'purchase_price_1day',
 'purchase_price_1person',
 'purchase_price_average',
 'purchase_price_category_average',
 'purchase_product_num',
 'ratio_category',
 'unit_price_category',
 'uuid',
 'visit_cycle',
 'visits_frequency']

In [8]:
train_data.head(11)

,uuid,coupon_id,coupon_use_result,fammily_num,child_num,income,visits_frequency,pass_days,all_purchase_price,purchase_num,...,coupon_use,purchase_price,unit_price_category,purchase_price_average,visit_cycle,ratio_category,adult_num,purchase_price_category_average,purchase_price_1person,purchase_price_1day
0,1,1,0,1,0,5,9,19,56562,1,...,0,534,267,6284,4,0.009441,1,267,56562,1346
1,1,2,0,1,0,5,9,19,56562,1,...,0,2803,1401,6284,4,0.049556,1,1401,56562,1346
2,1,3,0,1,0,5,9,19,56562,0,...,0,0,0,6284,4,0.000000,1,0,56562,1346
3,1,4,0,1,0,5,9,19,56562,1,...,0,764,764,6284,4,0.013507,1,764,56562,1346
4,1,5,0,1,0,5,9,19,56562,9,...,0,4981,293,6284,4,0.088063,1,293,56562,1346
5,1,6,0,1,0,5,9,19,56562,0,...,0,0,0,6284,4,0.000000,1,0,56562,1346
6,1,7,0,1,0,5,9,19,56562,0,...,0,0,0,6284,4,0.000000,1,0,56562,1346
7,1,8,0,1,0,5,9,19,56562,9,...,0,22620,293,6284,4,0.399915,1,293,56562,1346
8,1,9,0,1,0,5,9,19,56562,9,...,0,24860,2486,6284,4,0.439518,1,2486,56562,1346
9,1,10,0,1,0,5,9,19,56562,0,...,0,0,0,6284,4,0.000000,1,0,56562,1346


In [9]:
#LightGBMによるモデル作成
train_x = train_data.drop(['coupon_use_result','uuid'], axis=1)
train_y = train_data[['coupon_use_result']]
val_size = 4
uuid_num = train_data['uuid'].nunique()
train_uuid = int((uuid_num/val_size)*(val_size-1))
filter_idx = (train_uuid*11)-1
tr_x, va_x = train_x.iloc[:filter_idx], train_x.iloc[filter_idx:]
tr_y, va_y = train_y.iloc[:filter_idx], train_y.iloc[filter_idx:]

# params = {'objective': 'binary',
#          'metric': 'binary_logloss',
#          'seed': 70,
#          'feature_pre_filter': False,
#          'lambda_l1': 0.7363266301419099,
#          'lambda_l2': 0.0012895602337976008,
#          'num_leaves': 5,
#          'feature_fraction': 0.552,
#          'bagging_fraction': 0.5443702579911031,
#          'bagging_freq': 5,
#          'min_data_in_leaf': 20,
#          'min_child_samples': 20}
# params = {'objective': 'binary',
#          'metric': 'binary_logloss',
#          'seed': 71,
#          'feature_pre_filter': False,
#          'lambda_l1': 0.00000005204866804465978,
#          'lambda_l2': 0.00003133667635701077,
#          'num_leaves': 317,
#          'feature_fraction': 0.848640913268004,
#          'bagging_fraction': 0.4983467582046316,
#          'bagging_freq': 4,
#          'min_child_samples': 22}

score_dict = {}
# for i in range(200):
params = {'objective': 'binary',
         'metric': 'binary_logloss',
         'seed': 118,
         'lambda_l1': 0.00416566836241902,
         'lambda_l2': 0.10371698288325823,
         'num_leaves': 333,
         'feature_fraction': 0.6873251818776734,
         'bagging_fraction': 0.6363448284692337,
         'bagging_freq': 10,
         'min_child_samples': 17
        }
# params = {
# 'objective': 'binary',
#  'metric': 'binary_logloss',
#  'seed': 118,
#  'lambda_l1': 0.00004456120176148555,
#  'lambda_l2': 0.0000004244567009620525,
#  'num_leaves': 371,
#  'feature_fraction': 0.6045840940131413,
#  'bagging_fraction': 0.7154034241314151,
#  'bagging_freq': 6,
#  'min_child_samples': 58
#         }

num_round =1000
# categorical_feature = ['marry_flg','age']
scores = []

lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

best_params, tuning_history = dict(), list()
model = lgb.train(params, 
                  lgb_train, 
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
#                   categorical_feature = categorical_feature,
                  num_boost_round=num_round, 
                  early_stopping_rounds=100,
                  valid_names=['train', 'valid'], 
                  valid_sets=[lgb_train, lgb_eval]
                 )

va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
scores.append(score)
# score_dict[i] = score
#     print("="*80)

print("平均logloss：",np.mean(scores))

Training until validation scores don't improve for 100 rounds
[50]	train's binary_logloss: 0.00989969	valid's binary_logloss: 0.0595462
[100]	train's binary_logloss: 0.00254294	valid's binary_logloss: 0.0726514
Early stopping, best iteration is:
[13]	train's binary_logloss: 0.0378827	valid's binary_logloss: 0.0486878
平均logloss： 0.04868782829132765


In [10]:
score_dict

{}

In [11]:
scores

[0.04868782829132765]

In [12]:
#テストデータの予測
pred = model.predict(test_data.drop('uuid',axis=1))
pred = pd.Series(pred, name='pred')

In [13]:
# 特徴量重要度の算出 (データフレームで取得)
cols = list(train_x.columns)         # 特徴量名のリスト
f_importance = np.array(model.feature_importance()) # 特徴量重要度の算出
f_importance = f_importance / np.sum(f_importance)  # 正規化(必要ない場合はコメントアウト)
df_importance = pd.DataFrame({'feature':cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

,feature,importance
10,purchase_price,0.101770
6,all_purchase_price,0.088496
8,purchase_product_num,0.088496
11,unit_price_category,0.088496
5,pass_days,0.084071
17,purchase_price_1person,0.070796
0,coupon_id,0.061947
14,ratio_category,0.061947
4,visits_frequency,0.061947
12,purchase_price_average,0.061947


In [14]:
pred

0       0.003726
1       0.003726
2       0.003726
3       0.003726
4       0.010019
          ...   
3911    0.007258
3912    0.010778
3913    0.009631
3914    0.009631
3915    0.017302
Name: pred, Length: 3916, dtype: float64

In [15]:
def round_score(pred):
    if pred <= 0.0056:
        pred = 0
    elif pred >= 0.9944:
        pred = 1
    return pred

In [16]:
pred = pred.apply(lambda x: round_score(x))
result = test_data.join(pred)[['uuid', 'coupon_id', 'pred']]
result = result.pivot(index='uuid', columns='coupon_id', values='pred')

In [17]:
pred.min()

0.0

In [18]:
pred.max()

0.4040067200215342

In [19]:
result

coupon_id,1,2,3,4,5,6,7,8,9,10,11
uuid,,,,,,,,,,,
357,0.000000,0.000000,0.000000,0.000000,0.010019,0.000000,0.000000,0.019228,0.000000,0.000000,0.000000
358,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053306,0.038012,0.019076,0.019541
359,0.023098,0.011099,0.000000,0.007920,0.008600,0.000000,0.005723,0.404007,0.014478,0.000000,0.006285
360,0.000000,0.006747,0.000000,0.000000,0.000000,0.000000,0.000000,0.006771,0.000000,0.000000,0.000000
361,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012730,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
708,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029154,0.000000,0.000000,0.007670
709,0.005654,0.000000,0.000000,0.007445,0.000000,0.000000,0.007786,0.028291,0.016769,0.011618,0.012511
710,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [85]:
result.to_csv('/Users/atsushisato/git_clone/ai_quest_season2/models/model.csv',header=False)

In [31]:
pred.nunique()

1083

# optunaによるパラメータ調整

In [16]:
model.params

{'objective': 'binary',
 'metric': 'binary_logloss',
 'seed': 71,
 'feature_pre_filter': False,
 'lambda_l1': 1.0075794784727784e-08,
 'lambda_l2': 0.023033986719849988,
 'num_leaves': 317,
 'feature_fraction': 0.7,
 'bagging_fraction': 0.4983467582046316,
 'bagging_freq': 4,
 'min_data_in_leaf': 22,
 'min_child_samples': 20}

In [68]:
# パラメータ調整用

# prediction = np.rint(model.predict(va_x, num_iteration=model.best_iteration))
# accuracy = accuracy_score(va_y, prediction)

# best_params = model.params
# print("Best params:", display(best_params)
# print("  Accuracy = {}".format(accuracy))
# print("  Params: ")
# for key, value in best_params.items():
#     print("    {}: {}".format(key, value))

In [69]:
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.05,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'seed': 0,
        'verbosity': -1,
    }
    gbm = lgb.train(param, 
                    lgb_train, 
                    valid_sets=lgb_eval,
                    verbose_eval=False, 
                    num_boost_round=num_round, 
                    early_stopping_rounds=100
                   )
    y_prob = gbm.predict(va_x)
    y_pred = np.round(y_prob)
    return roc_auc_score(
        np.round(va_y.values),
        np.round(y_pred)
    )

In [70]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2021-01-05 15:27:25,839] A new study created in memory with name: no-name-9513d28e-a180-4b79-a86a-80ca7cedfb9e
[I 2021-01-05 15:27:25,988] Trial 0 finished with value: 0.5 and parameters: {'lambda_l1': 4.456120176148555e-05, 'lambda_l2': 4.244567009620525e-07, 'num_leaves': 371, 'feature_fraction': 0.6045840940131413, 'bagging_fraction': 0.7154034241314151, 'bagging_freq': 6, 'min_child_samples': 58}. Best is trial 0 with value: 0.5.
[I 2021-01-05 15:27:26,183] Trial 1 finished with value: 0.5 and parameters: {'lambda_l1': 5.49980142712909e-07, 'lambda_l2': 6.519782022908096e-08, 'num_leaves': 284, 'feature_fraction': 0.8218208360322309, 'bagging_fraction': 0.5980894860781979, 'bagging_freq': 0, 'min_child_samples': 17}. Best is trial 0 with value: 0.5.
[I 2021-01-05 15:27:26,273] Trial 2 finished with value: 0.5 and parameters: {'lambda_l1': 1.9199684319685696e-05, 'lambda_l2': 6.590368737657856e-05, 'num_leaves': 99, 'feature_fraction': 0.7496063053146955, 'bagging_fraction': 0.52

In [71]:
#ベストパラメータ
print('Number of finished trials:', len(study.trials))
#Number of finished trials: 50


Number of finished trials: 50


In [72]:
print('Best trial:', display(study.best_trial.params))

#初回
# {'lambda_l1': 5.204866804465978e-08,
#  'lambda_l2': 3.133667635701077e-05,
#  'num_leaves': 317,
#  'feature_fraction': 0.848640913268004,
#  'bagging_fraction': 0.4983467582046316,
#  'bagging_freq': 4,
#  'min_child_samples': 22}
# Best trial: None

#2回目
# {'lambda_l1': 0.00416566836241902,
#  'lambda_l2': 0.10371698288325823,
#  'num_leaves': 333,
#  'feature_fraction': 0.6873251818776734,
#  'bagging_fraction': 0.6363448284692337,
#  'bagging_freq': 10,
#  'min_child_samples': 17}
# Best trial: None

{'lambda_l1': 4.456120176148555e-05,
 'lambda_l2': 4.244567009620525e-07,
 'num_leaves': 371,
 'feature_fraction': 0.6045840940131413,
 'bagging_fraction': 0.7154034241314151,
 'bagging_freq': 6,
 'min_child_samples': 58}

Best trial: None


In [ ]:
# def create_feature_together(train, test, num_cols):
#     train, test = transform_box_cox(train, test, num_cols)
#     return train, test

# # issue 14 Box-Cox変換
# def transform_box_cox(train, test, column):
#     pt = PowerTransformer(method='box-cox')
#     pt.fit(train[column])

#     # 変換後のデータで各列を置換
#     train[column] = pt.transform(train[column])
#     test[column] = pt.transform(test[column])

#特徴量作成（train_dataとtest_dataで同一の処理が必要）
#変換するカラム
# num_cols = ['all_purchase_price']
# train_data, test_data = create_feature_together(train_data, test_data, num_cols)

In [ ]:
# tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y,
#                                           test_size=0.25, random_state=71, shuffle=True)
# kf = KFold(n_splits=4, shuffle=True, random_state=71)
# for tr_idx, va_idx in kf.split(train_x):
#     tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
#     tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]